In [1]:
import os,sys,subprocess,time
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
pio.templates["mod"] = go.layout.Template(layout=dict(font=dict(family="Fira Code")))
pio.templates.default = "plotly_dark+mod"
from zipfile import ZipFile
from glob import glob
from sklearn.preprocessing import StandardScaler,OrdinalEncoder,MinMaxScaler,RobustScaler,OneHotEncoder
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV,train_test_split,StratifiedShuffleSplit
from sklearn.metrics import roc_auc_score,confusion_matrix,accuracy_score,f1_score,precision_recall_curve
import miceforest as mf
import tensorflow as tf
from tensorflow import keras
tf.get_logger().setLevel('ERROR')
%matplotlib inline

In [2]:
peng_lter = pd.read_csv('penguins_lter.csv')

In [3]:
peng_lter.head(3)

,studyName,Sample Number,Species,Region,Island,Stage,Individual ID,Clutch Completion,Date Egg,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Body Mass (g),Sex,Delta 15 N (o/oo),Delta 13 C (o/oo),Comments
0,PAL0708,1,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N1A1,Yes,11/11/07,39.1,18.7,181.0,3750.0,MALE,NaN,NaN,Not enough blood for isotopes.
1,PAL0708,2,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N1A2,Yes,11/11/07,39.5,17.4,186.0,3800.0,FEMALE,8.94956,-24.69454,NaN
2,PAL0708,3,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N2A1,Yes,11/16/07,40.3,18.0,195.0,3250.0,FEMALE,8.36821,-25.33302,NaN


In [4]:
peng_lter.shape

(344, 17)

In [5]:
peng_lter.describe()

,Sample Number,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Body Mass (g),Delta 15 N (o/oo),Delta 13 C (o/oo)
count,344.000000,342.000000,342.000000,342.000000,342.000000,330.000000,331.000000
mean,63.151163,43.921930,17.151170,200.915205,4201.754386,8.733382,-25.686292
std,40.430199,5.459584,1.974793,14.061714,801.954536,0.551770,0.793961
min,1.000000,32.100000,13.100000,172.000000,2700.000000,7.632200,-27.018540
25%,29.000000,39.225000,15.600000,190.000000,3550.000000,8.299890,-26.320305
50%,58.000000,44.450000,17.300000,197.000000,4050.000000,8.652405,-25.833520
75%,95.250000,48.500000,18.700000,213.000000,4750.000000,9.172123,-25.062050
max,152.000000,59.600000,21.500000,231.000000,6300.000000,10.025440,-23.787670


In [6]:
peng_lter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   studyName            344 non-null    object 
 1   Sample Number        344 non-null    int64  
 2   Species              344 non-null    object 
 3   Region               344 non-null    object 
 4   Island               344 non-null    object 
 5   Stage                344 non-null    object 
 6   Individual ID        344 non-null    object 
 7   Clutch Completion    344 non-null    object 
 8   Date Egg             344 non-null    object 
 9   Culmen Length (mm)   342 non-null    float64
 10  Culmen Depth (mm)    342 non-null    float64
 11  Flipper Length (mm)  342 non-null    float64
 12  Body Mass (g)        342 non-null    float64
 13  Sex                  334 non-null    object 
 14  Delta 15 N (o/oo)    330 non-null    float64
 15  Delta 13 C (o/oo)    331 non-null    flo

As comments column is heavily downsized and missing we can omit it<br>
As this is heavily specie-related dataset lets clean the species column first<br>
since there is only one Region we can drop it<br>
since there is only one Stage we can drop it

In [23]:
def preprocess(data:pd.DataFrame):
   df = data.drop(["Comments"],axis=1)
   df["Species"] = df.Species.str.split(n=1,expand=True)[0]
   df.drop(columns=['Region'],inplace=True)
   df.drop(columns=['Stage'],inplace=True)
   df[['N','A']] =df['Individual ID'].str.split(r'(\d+)',expand=True)[[1,3]]
   df['Date Egg'] = pd.to_datetime(df['Date Egg'],format='mixed')
   df.drop(columns=['Individual ID'],inplace=True)
   original_columns =df.columns.to_list()
   df.columns = ['study','samples','species','island','clutch','date','culmenL','culmenD','flipperL','bmass','sex','delta_15n','delta_13c','N','A']
   df[['N','A']] =df[['N','A']].astype(np.float32)
   return df

<font size=4>

|Column Name|Description|Type|
|:----------|:----------|:---|
|StudyName|Sampling expedition from which data were collected, generated, etc.|categorical|
|SampleNumber|an integer denoting the continuous numbering sequence for each sample|categorical|
|Species|a character string denoting the penguin species|categorical|
|Region|a character string denoting the region of Palmer LTER sampling grid|categorical|
|Island|a character string denoting the island near Palmer Station where samples were collected|categorical|
|Stage|a character string denoting reproductive stage at sampling|categorical|
|IndividualID|a character string denoting the unique ID for each individual in dataset|categorical|
|Clutch Completion|a character string denoting if the study nest observed with a full clutch, i.e., 2 eggs|categorical|
|Date Egg|a date denoting the date study nest observed with 1 egg (sampled)|continuous|
|Culmen Length|a number denoting the length of the dorsal ridge of a bird's bill (millimeters)|continuous|
|Culmen Depth|a number denoting the depth of the dorsal ridge of a bird's bill (millimeters)|continuous|
|Flipper Length|an integer denoting the length penguin flipper (millimeters)|continuous|
|Body Mass|an integer denoting the penguin body mass (grams)|continuous|
|Sex|a character string denoting the sex of an animal|categorical|
|Delta 15 N|a number denoting the measure of the ratio of stable isotopes 15N:14N|continuous|
|Delta 13 C|a number denoting the measure of the ratio of stable isotopes 13C:12C|continuous|
|N|Individual Number assigned to penguin|categorical|
|A|Individual Number assigned to penguin|categorical|

In [24]:
temp_df = preprocess(peng_lter)

In [25]:
temp_df.head()

,study,samples,species,island,clutch,date,culmenL,culmenD,flipperL,bmass,sex,delta_15n,delta_13c,N,A
0,PAL0708,1,Adelie,Torgersen,Yes,2007-11-11,39.1,18.7,181.0,3750.0,MALE,NaN,NaN,1.0,1.0
1,PAL0708,2,Adelie,Torgersen,Yes,2007-11-11,39.5,17.4,186.0,3800.0,FEMALE,8.94956,-24.69454,1.0,2.0
2,PAL0708,3,Adelie,Torgersen,Yes,2007-11-16,40.3,18.0,195.0,3250.0,FEMALE,8.36821,-25.33302,2.0,1.0
3,PAL0708,4,Adelie,Torgersen,Yes,2007-11-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0
4,PAL0708,5,Adelie,Torgersen,Yes,2007-11-16,36.7,19.3,193.0,3450.0,FEMALE,8.76651,-25.32426,3.0,1.0


In [26]:
temp_na = temp_df.dropna()

# Univariate Analysis

## Continuous

### Date

In [41]:
pd.DatetimeIndex(temp_na.date).year.unique()

Index([2007, 2008, 2009], dtype='int32', name='date')

In [33]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=temp_na.date,y=[1]*temp_na.shape[0],mode="markers"))
fig.show()

In [ ]:
# ord_enc = OrdinalEncoder().set_output(transform='pandas')
# temp_df[['study','species','island','clutch','sex']] = ord_enc.fit_transform(temp_df[['study','species','island','clutch','sex']])
# temp_df.head()
# kds = mf.ImputationKernel(data=temp_df.drop(columns=['date']),datasets=5,random_state=1991)
# kds.mice(1)
# plt.rcParams["figure.figsize"] = (20,10)
# kds.plot_imputed_distributions(datasets=1)
# temp_df = kds.complete_data(dataset=1)
# GradientBoostingClassifier().get_params()
# gird_params = dict(
#     learnig_rate=np.logspace(-6,-1,6),
#     max_depth=np.arange(3,16),
#     max_leaf_nodes=np.arange(8,32),
#     n_estimators=np.arange(100,500,50)
#     )